### Content
    0. File Path
    1. Load Vertices
    2. Load Edges
    3. Create Graph & Test
    4. GST Requirements
    5. GST Steps
    6. BT-QA main_GST Steps ( Plan )

## 4. GST Requirements
### Function: 
    call_main_GST
    (QKG_file, cornerstone_file, qtype, answer_list_file, no_GST, gdict, verbose, gt,config,con)
    
### Corresponding Parameter
    f1, f2, f4, f5, no_GST, gdict, verbose, gt1, config, context_file
        
    0. QKG_file            f1=argv[1] #input QKG
    1. cornerstone_file    f2=argv[2] #input Cornerstones 
    2. qtype               f4=argv[3] #answer match / qtype
    3. answer_list_file    f5=argv[4] #answer type /answer_list_file 
    4. no_GST              no_GST=int(argv[5]) #number of GSTs
    5. gdict               gdict
    6. verbose             verbose=int(argv[6])
    7. gt
    8. config
    9. con                 context file
    
### Corresponding Value
    0. f1 = "./files/QKG_ques-q1"  
        *** What: KG 
        *** From: Generated by Data2Graph
        *** How to use it in GST:
        
    1. f2 = "./files/QKG_cornerstones_ques-q1" 
        *** What: KG Gpickle 
        *** From: Generated by Data2Graph, cornerstones?? and them selected by ????
        *** How to use it in GST:
        
    2. f4 = "question_type.txt" 
        *** What: question type
        *** From: get_answer_types_from_questions.py
        *** How to use it in GST:used to check the answer type and select related answer
        
    3. f5 = "./files/Answer_list_ques-q1" 
        
        *** What: answer_list_file='./files/Answer_list_'+str(qid) only represent an address
        *** From: path parameter
        *** How to use it in GST: used to named answer-related address
    
    4. no_GST = 50 #Number of GSTs to be considered
        *** What: 
        *** From: config
        *** How to use it in GST:set the limited node number  
        
    5. gdict = 
            # gdict={}
            # model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)  
            # word_vectors = model.wv
            # gdict=word_vectors
            
        *** What: word2vector
        *** From: gensim.models.KeyedVectors
        *** How to use it in GST: for calculate the Predicate weight(no need to used in BT-QA)

    6. verbose=int(argv[6])      
        *** What: To print intermediate statements in verbose manner make it 1, other wise 0 
        *** From: Config
        *** How to use it in GST:
    
    7. gt = set(['unknown'])
        *** What: a set of answer, the initial is 'unknown'
        *** From: 
        original from reformat_dataset_single.py
        in get_formatted_docs function,  
        fp.write('Answer\t' + 'Unknown\n')
        *** How to use it in GST:
        
    8. config stream = open("config.yml", 'r')
        for line in stream:
        if not line.startswith('#') and len(line)>1:
            line=line.split(':')
            config[line[0]]=line[1].strip()
       
    9. con = "./files/context_ques-q1"  
        *** What: context_file, not be used in GST, should be deleted

## 5. GST Steps
    - 0. load graph & corner
    - 1. initialize_queue 
        (combine the 'corner' with graph and get edges' cost between 'corner node' and 'question node' in graph. see the result T & Q)
    - 2. get GST set
    - 3. get answer type 
    - 4. trees with potential answers by filtering answer type from GST set
    - 5. answering merge (need to be changed because mention node provide more convience)
    - 6. find unique sets of answers and get result list

## 6. BT-QA main_GST Steps ( Plan )
### preprepare (libraries part)
- 0. For graph part: 
    - add node property (only for 'entity' & 'mention' nodes): weight & matched 
         - reference: "def add_node_weights" in "generate_graphs_from_triples.py"
        
    - add edges property wight & wlist 
         - reference: "def distribute_node_weights" in "generate_graphs_from_triples.py"
        
    - add type node of all 'mention' node
         - reference: "def add_type_edges" in "generate_graphs_from_triples.py"
        
    - change the id from 'number' to 'vocabulary: Node Type(doc/sent/clause/entity/mention/type)'
        
    - Structure:
   
            *Nodes:
            - ID:'vocabulary: Node Type(doc/sent/clause/entity/mention/type)'
            - Attribute:
                (u“id”, {
                ‘type’:“”
                ‘attr1’:“”
                ‘attr2’:“”
                ‘attr3’:“”
                ‘matched’:“” ##for matched questions' entities
                ‘weight’:0.0 ##for matched weight between node and questions' entities
                })
            - the weight are calculted by ‘Jaccard Coefficient’

            *Edges:
            - Node1, Node2
            - Attribute:
            (u'node1’,
             u'node2',
             {'weight': 0.0,
              'wlist': [0.0，0.0]})

    - generate corner and save it into pickle 
        reference: "generate_graphs_from triples in generate graph from triples.py"


- 1. Generate answer type from question
     - reference: get_answer_types_from_questions.py
    
### steps: ( 24 functions need to be modified, 14 of them contains graph )
- 0. load graph & corner
- 1. initialize_queue (2 def)
    (combine the 'corner' with graph and get edges' cost between 'corner node' and 'question node' in graph. see the result T & Q)
- 2. get GST set (6 def)
- 3. get answer type 
- 4. trees with potential answers by filtering answer type from GST set (4 def)
- 5. answering merge (need to be changed because Entity node provide more convience) (2 def)
- 6. find unique sets of answers and get result list (10 def)
    
    future plan: 
    I'll try to extract predicate and generate predicate node from clause to see if the precision will be enhanced or not.

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
sc.addPyFile('/usr/local/Cellar/apache-spark/2.4.1/libexec/jars/graphframes-0.7.0-spark2.4-s_2.11.jar')

In [11]:
#1. RDD -> DataFrame -> GraphFrame (code need to be change when add attr)
#2. GraphX -> GraphFrame (code no need to change when add attr) encounter problem：https://github.com/graphframes/graphframes/issues/359

## 7. Code part
### 7.1 preprepare (libraries part)
### 7.1.1 modify the id from "number" to "nodeType:Attr1:id" (1.edges, 2.nodes)

#### 1.1.1 get answer type from quetion

In [5]:
# modules
import sys
from graphframes import *
import pyspark.sql.functions as f

# library 
sys.path.append("project/library")
import get_answer_types_from_questions as gtQType
import get_all_subjects_predicates_from_questions as gtQSub
from stanfordcorenlp import StanfordCoreNLP

In [9]:
verbose = 0

In [7]:
nlp = StanfordCoreNLP(r'/Users/jeanxu/PycharmProjects/QUEST_TerminalTest/Code/stanford-corenlp-full-2018-10-05')

In [17]:
#Prune the predicate terms from the query if the query has more than 'prune' number of terms
prune=5

In [10]:
question = 'when is national day of England '

In [21]:
#Extracting potential answer type from question
f1='question_type.txt'
gtQType.get_answer_type_main(question,f1,nlp)

In [22]:
#Extracting query terms from question.
s1='question_subject_predicate.txt'
gtQSub.get_sub_pred_ques_main(question,s1,nlp)

In [23]:
aux_list=set()
for l in open('auxiliary_verb_list.txt'):
    l=l.strip()
    aux_list.add(l.lower())

In [77]:
aux_list

{'am',
 'are',
 'be',
 'been',
 'being',
 'can',
 'could',
 'dare',
 'did',
 'do',
 'done',
 'had',
 'has',
 'have',
 'is',
 'may',
 'might',
 'must',
 'need',
 'ought',
 'shall',
 'should',
 'was',
 'were',
 'will',
 'would'}

In [25]:
# Read entities of question
s11=open(s1, 'r')
q_ent = set()
type_qent = {}
for line in s11:
    line = line.strip()
    line = line.split()
    s = line[0]
    for i in range(1, len(line) - 1):
        s += ' ' + line[i]
    q_ent.add(s.lower())
    type_qent[s.lower()] = line[len(line) - 1]
if verbose:
    print "Query terms ->", len(q_ent), q_ent
if len(q_ent) > prune:
    if verbose:
        print "Pruning.."
    p = frozenset(q_ent)
    for s in p:
        if type_qent[s] == 'P':
            q_ent.remove(s)

if verbose:
    print "Without Predicate Query terms ->", len(q_ent), q_ent

p = frozenset(q_ent)
for s in p:
    if s in aux_list and type_qent[s] == 'P':
        q_ent.remove(s)

if verbose:
    print "Without Auxiliary Query terms ->", len(q_ent), q_ent

In [27]:
type_qent

{'england national day': 'NE', 'is': 'P'}

In [26]:
q_ent

{'england national day'}

## 0. File Path

In [58]:
File_verticesTextRDD = "/Users/jeanxu/Documents/UniLU/0_MasterThesis/4_Spark/ReferenceFromPaul/Graph-Small/Vertices/part-*"

In [59]:
File_edgesTextRDD = "/Users/jeanxu/Documents/UniLU/0_MasterThesis/4_Spark/ReferenceFromPaul/Graph-Small/Edges/part-*"

## 1. Load Vertices

In [60]:
verticesText0 = spark.read.csv(File_verticesTextRDD, header='false', inferSchema='false', sep='\t')
#rdd=sc.textFile(File_verticesTextRDD).map(lambda line : line.split('\t')).toDF()
#rdd.createOrReplaceTempView("vertices")
#rdd.show()

In [61]:
verticesText0.show(10)
# Document：(id, DocumentProperty(title, url, domain, tweets))
# Sentence：(id, SentenceProperty(sentenceNumber, sentenceContent, root, links.toList(lenth!=0)))
# Clause：  (id, ClauseProperty(clause,clauseType,subject,relation,argument,root,additional.toList))
# Mention： (id, MentionProperty(mention, NERtype, entity))
# Entity：  (id, EntityProperty(entity))

+--------+---+---+--------------------+-------------------+------+
|     _c0|_c1|_c2|                 _c3|                _c4|   _c5|
+--------+---+---+--------------------+-------------------+------+
|Sentence|  0|  1|Politics of Burun...|Politics of Burundi| $END$|
|Sentence|  1|  2|Executive power i...|Politics of Burundi| $END$|
|Sentence|  2|  3|Legislative power...|Politics of Burundi| $END$|
|Sentence|  3|  4|The political lan...|Politics of Burundi| $END$|
|Sentence|  4|  5|The current Presi...|Politics of Burundi| $END$|
|Sentence|  5|  6|Nkurunziza was th...|Politics of Burundi| $END$|
|Sentence|  6|  7|In November 1995,...|Politics of Burundi| $END$|
|Sentence|  7|  8|In July 1996, for...|Politics of Burundi| $END$|
|Sentence|  8|  9|He declared himse...|Politics of Burundi| $END$|
|Sentence|  9| 10|Widespread condem...|Politics of Burundi| $END$|
+--------+---+---+--------------------+-------------------+------+
only showing top 10 rows



In [ ]:
##exmaple:
# Document	5691614	Office of the United Nations High Commissioner for Human Rights			List() $END$
# Document	5691615	Gecko			List() $END$
# Document	5691616	Maeshowe			List() $END$
# Document	5691617	Kiwi			List() $END$
# Document	5691618	Iblis			List() $END$
# Document	5691619	Number			List() $END$

# Sentence	0	1	Politics of Burundi takes place in a framework of a transitional presidential representative democratic republic, whereby the President of Burundi is both head of state and head of government, and of a multi-party system.	Politics of Burundi	 $END$
# Sentence	1	2	Executive power is exercised by the government.	Politics of Burundi	 $END$

# Clause	24971	("This method", "was used", "by an estimated 38 million couples worldwide", "in 1991")	SVA	This method	was used	by an estimated 38 million couples worldwide	Coitus interruptus-3	in 1991		 $END$
# Clause	24972	("what", "was transpiring", "quickly", "spread and everybody")	UNKNOWN	what	was transpiring	quickly	First Council of Constantinople-36	spread and everybody		 $END$
# Clause	24973	("The news of what was transpiring quickly spread and everybody", "rushed", "to the church")	SVA	The news of what was transpiring quickly spread and everybody	rushed	to the church	First Council of Constantinople-36		 $END$

# Mention	24975	Freedom House	MISC	Freedom_House $END$
# Mention	24976	NGO	ORGANIZATION	Non-governmental_organization $END$
# Mention	24977	Obiang	PERSON	Teodoro_Obiang_Nguema_Mbasogo $END$

# Entity	8593089	Josephine_of_Leuchtenberg $END$
# Entity	8593090	Public_Auditorium $END$
# Entity	8593091	West_Plains,_Missouri $END$

![jupyter](./GraphExample.png)

In [121]:
verticesText1 = verticesText0.select("_c1","_c0","_c2","_c3","_c4")\
.withColumnRenamed("_c0", "nodeType").withColumnRenamed("_c1", "id")\
.withColumnRenamed("_c2", "attr1").withColumnRenamed("_c3", "attr2")\
.withColumnRenamed("_c4", "attr3")

In [122]:
verticesText1.show(10)

+---+--------+-----+--------------------+-------------------+
| id|nodeType|attr1|               attr2|              attr3|
+---+--------+-----+--------------------+-------------------+
|  0|Sentence|    1|Politics of Burun...|Politics of Burundi|
|  1|Sentence|    2|Executive power i...|Politics of Burundi|
|  2|Sentence|    3|Legislative power...|Politics of Burundi|
|  3|Sentence|    4|The political lan...|Politics of Burundi|
|  4|Sentence|    5|The current Presi...|Politics of Burundi|
|  5|Sentence|    6|Nkurunziza was th...|Politics of Burundi|
|  6|Sentence|    7|In November 1995,...|Politics of Burundi|
|  7|Sentence|    8|In July 1996, for...|Politics of Burundi|
|  8|Sentence|    9|He declared himse...|Politics of Burundi|
|  9|Sentence|   10|Widespread condem...|Politics of Burundi|
+---+--------+-----+--------------------+-------------------+
only showing top 10 rows



### 1.1 add node attributes 

In [71]:
from pyspark.sql.functions import lit

In [68]:
# verticesText2=verticesText1.withColumn("did", "").withColumn("dtitle", "").withColumn("sid", "").withColumn("weight", 0.0).withColumn("matched", "")

In [77]:
# verticesText1=verticesText1.withColumn("did", lit(0)).withColumn("dtitle", lit("")).withColumn("sid", lit(0)).withColumn("weight", lit(0)).withColumn("matched", lit(""))

In [123]:
# verticesText1.show()

#### document

In [172]:

doc_verticesText=verticesText1.filter("nodeType = 'Document'")

In [173]:
doc_verticesText.show()

+----+--------+--------------------+-----+-----+
|  id|nodeType|               attr1|attr2|attr3|
+----+--------+--------------------+-----+-----+
|4526|Document|     Africa Alphabet| null| null|
|4527|Document|               Brain| null| null|
|4528|Document|     Andreas Aagesen| null| null|
|4529|Document|Australian Broadc...| null| null|
|4530|Document|The Elephant 6 Re...| null| null|
|4531|Document|Albert of Branden...| null| null|
|4532|Document|Telecommunication...| null| null|
|4533|Document|                 DMA| null| null|
|4534|Document|         Alford plea| null| null|
|4535|Document|        Claude Piron| null| null|
|4536|Document|Full disclosure (...| null| null|
|4537|Document|    Anthem of Europe| null| null|
|4538|Document|             Cytosol| null| null|
|4539|Document|            Barbagia| null| null|
|4540|Document|             Calcium| null| null|
|4541|Document|   Chomsky hierarchy| null| null|
|4542|Document| Critical philosophy| null| null|
|4543|Document|   Cu

In [174]:
#add attributes--documnets
doc_verticesText=doc_verticesText.withColumn("did", doc_verticesText.id).withColumn("dtitle", doc_verticesText.attr1).drop("attr1").drop("attr2").drop("attr3")

In [175]:
doc_verticesText.show()

+----+--------+----+--------------------+
|  id|nodeType| did|              dtitle|
+----+--------+----+--------------------+
|4526|Document|4526|     Africa Alphabet|
|4527|Document|4527|               Brain|
|4528|Document|4528|     Andreas Aagesen|
|4529|Document|4529|Australian Broadc...|
|4530|Document|4530|The Elephant 6 Re...|
|4531|Document|4531|Albert of Branden...|
|4532|Document|4532|Telecommunication...|
|4533|Document|4533|                 DMA|
|4534|Document|4534|         Alford plea|
|4535|Document|4535|        Claude Piron|
|4536|Document|4536|Full disclosure (...|
|4537|Document|4537|    Anthem of Europe|
|4538|Document|4538|             Cytosol|
|4539|Document|4539|            Barbagia|
|4540|Document|4540|             Calcium|
|4541|Document|4541|   Chomsky hierarchy|
|4542|Document|4542| Critical philosophy|
|4543|Document|4543|   Cultural movement|
|4544|Document|4544|          Braveheart|
|4545|Document|4545|                Bill|
+----+--------+----+--------------

#### sentence

In [158]:
## sentence
sen_verticesText=verticesText1.filter("nodeType = 'Sentence'")

In [159]:
sen_verticesText.show()

+---+--------+-----+--------------------+-------------------+
| id|nodeType|attr1|               attr2|              attr3|
+---+--------+-----+--------------------+-------------------+
|  0|Sentence|    1|Politics of Burun...|Politics of Burundi|
|  1|Sentence|    2|Executive power i...|Politics of Burundi|
|  2|Sentence|    3|Legislative power...|Politics of Burundi|
|  3|Sentence|    4|The political lan...|Politics of Burundi|
|  4|Sentence|    5|The current Presi...|Politics of Burundi|
|  5|Sentence|    6|Nkurunziza was th...|Politics of Burundi|
|  6|Sentence|    7|In November 1995,...|Politics of Burundi|
|  7|Sentence|    8|In July 1996, for...|Politics of Burundi|
|  8|Sentence|    9|He declared himse...|Politics of Burundi|
|  9|Sentence|   10|Widespread condem...|Politics of Burundi|
| 10|Sentence|   11|Buyoya agreed in ...|Politics of Burundi|
| 11|Sentence|   12|Nonetheless, figh...|Politics of Burundi|
| 12|Sentence|   13|In June 1998, Buy...|Politics of Burundi|
| 13|Sen

In [166]:
sen_verticesText.count()

1411580

In [161]:
sen_verticesTextJ=sen_verticesText.join(doc_verticesText.select("did","dtitle"),sen_verticesText.attr3==doc_verticesText.dtitle, "inner")

In [162]:
sen_verticesTextJ.show()

+-------+--------+-----+--------------------+--------------------+-------+--------------------+
|     id|nodeType|attr1|               attr2|               attr3|    did|              dtitle|
+-------+--------+-----+--------------------+--------------------+-------+--------------------+
|7282397|Sentence|    1|   A "Hello, World!"|&quot;Hello, Worl...|5691624|&quot;Hello, Worl...|
|7282398|Sentence|    2|program generally...|&quot;Hello, Worl...|5691624|&quot;Hello, Worl...|
|7282399|Sentence|    3|                   .|&quot;Hello, Worl...|5691624|&quot;Hello, Worl...|
|7282400|Sentence|    4|Because it is ver...|&quot;Hello, Worl...|5691624|&quot;Hello, Worl...|
|7282401|Sentence|    5|   A "Hello, World!"|&quot;Hello, Worl...|5691624|&quot;Hello, Worl...|
|7282402|Sentence|    6|program is tradit...|&quot;Hello, Worl...|5691624|&quot;Hello, Worl...|
|7282403|Sentence|    7|       Hello, world!|&quot;Hello, Worl...|5691624|&quot;Hello, Worl...|
|7282404|Sentence|    8|is also traditio

In [165]:
sen_verticesTextJ.count()

1410800

In [167]:
sen_verticesText=sen_verticesTextJ.withColumn("sid", sen_verticesTextJ.attr1).drop("attr1").drop("attr2").drop("attr3")

In [168]:
sen_verticesText.show()

+-------+--------+-------+--------------------+---+
|     id|nodeType|    did|              dtitle|sid|
+-------+--------+-------+--------------------+---+
|7282397|Sentence|5691624|&quot;Hello, Worl...|  1|
|7282398|Sentence|5691624|&quot;Hello, Worl...|  2|
|7282399|Sentence|5691624|&quot;Hello, Worl...|  3|
|7282400|Sentence|5691624|&quot;Hello, Worl...|  4|
|7282401|Sentence|5691624|&quot;Hello, Worl...|  5|
|7282402|Sentence|5691624|&quot;Hello, Worl...|  6|
|7282403|Sentence|5691624|&quot;Hello, Worl...|  7|
|7282404|Sentence|5691624|&quot;Hello, Worl...|  8|
|7282405|Sentence|5691624|&quot;Hello, Worl...|  9|
|7282406|Sentence|5691624|&quot;Hello, Worl...| 10|
|7282407|Sentence|5691624|&quot;Hello, Worl...| 11|
|7282408|Sentence|5691624|&quot;Hello, Worl...| 12|
|7282409|Sentence|5691624|&quot;Hello, Worl...| 13|
|7282410|Sentence|5691624|&quot;Hello, Worl...| 14|
|7282411|Sentence|5691624|&quot;Hello, Worl...| 15|
|7282412|Sentence|5691624|&quot;Hello, Worl...| 16|
|7282413|Sen

#### clause

In [203]:
##clause 
clause_verticesText=verticesText1.filter("nodeType = 'Clause'")

In [204]:
clause_verticesText.show()

+----+--------+--------------------+-----+--------------------+
|  id|nodeType|               attr1|attr2|               attr3|
+----+--------+--------------------+-----+--------------------+
|4559|  Clause|("A significant c...|  SVA|A significant con...|
|4560|  Clause|("by the French r...|  SVO|by the French res...|
|4561|  Clause|("quinine and str...|  SVC|quinine and stryc...|
|4562|  Clause|("strychnine", "i...|  SVC|          strychnine|
|4563|  Clause|("Some writers", ...|  SVO|        Some writers|
|4564|  Clause|("those", "would ...|  SVO|               those|
|4565|  Clause|("The English Cha...|  SVC|The English Chann...|
|4566|  Clause|("Johnston", "res...|  SVO|            Johnston|
|4567|  Clause|("Tupper", "becam...|  SVO|              Tupper|
|4568|  Clause|("His father", "m...|  SVA|          His father|
|4569|  Clause|("Many physicists...|  SVA|     Many physicists|
|4570|  Clause|("of quantum mech...|  SVC|of quantum mechanics|
|4571|  Clause|("The King", "app...|  SV

In [177]:
edgesText3.filter("label = 'contains the clause'").show()

+-------+-------+-------------------+-------+------------+-------+------------+
|    src|    dst|              label| src_id|src_nodeType| dst_id|dst_nodeType|
+-------+-------+-------------------+-------+------------+-------+------------+
|1234113| 100010|contains the clause|1234113|    Sentence| 100010|      Clause|
|3875588|1000240|contains the clause|3875588|    Sentence|1000240|      Clause|
|3650624|1000280|contains the clause|3650624|    Sentence|1000280|      Clause|
|2068926|1000665|contains the clause|2068926|    Sentence|1000665|      Clause|
| 467648|1000795|contains the clause| 467648|    Sentence|1000795|      Clause|
| 233944|1000839|contains the clause| 233944|    Sentence|1000839|      Clause|
|4022932|1000888|contains the clause|4022932|    Sentence|1000888|      Clause|
|1212133| 100140|contains the clause|1212133|    Sentence| 100140|      Clause|
| 452611|1001866|contains the clause| 452611|    Sentence|1001866|      Clause|
|1487892|1002011|contains the clause|148

In [206]:
clause_verticesTextJ=clause_verticesText.join(edgesText3.select("src_id","src_nodeType","dst_id","dst_nodeType"), clause_verticesText.id==edgesText3.dst_id, "inner")

In [207]:
clause_verticesTextJ=clause_verticesTextJ.drop("src_nodeType").drop("dst_id").drop("dst_nodeType")

In [218]:
clause_verticesTextJ.show()

+-------+--------+--------------------+-----+--------------------+-------+
|     id|nodeType|               attr1|attr2|               attr3| src_id|
+-------+--------+--------------------+-----+--------------------+-------+
| 100010|  Clause|("Kerensky", "ord...|  SVO|            Kerensky|1234113|
|1000240|  Clause|("to the paternal...|  SVC|to the paternal h...|3875588|
|1000280|  Clause|("roughly speakin...|  SVO|roughly speaking ...|3650624|
|1000665|  Clause|("sympatric speci...|  SVC|sympatric speciation|2068926|
|1000795|  Clause|("The growth of t...|  SVO|The growth of the...| 467648|
|1000839|  Clause|("the priests", "...|  SVA|         the priests| 233944|
|1000888|  Clause|("the absolute va...|  SVC|  the absolute value|4022932|
| 100140|  Clause|("A memorial plaq...|  SVA|   A memorial plaque|1212133|
|1001866|  Clause|("1QDan", "is", "...|  SVC|               1QDan| 452611|
|1002011|  Clause|("the use of Afri...|  SVC|the use of Africa...|1487892|
|1002185|  Clause|("he", 

In [185]:
sen_verticesText.show()

+-------+--------+-------+--------------------+---+
|     id|nodeType|    did|              dtitle|sid|
+-------+--------+-------+--------------------+---+
|7282397|Sentence|5691624|&quot;Hello, Worl...|  1|
|7282398|Sentence|5691624|&quot;Hello, Worl...|  2|
|7282399|Sentence|5691624|&quot;Hello, Worl...|  3|
|7282400|Sentence|5691624|&quot;Hello, Worl...|  4|
|7282401|Sentence|5691624|&quot;Hello, Worl...|  5|
|7282402|Sentence|5691624|&quot;Hello, Worl...|  6|
|7282403|Sentence|5691624|&quot;Hello, Worl...|  7|
|7282404|Sentence|5691624|&quot;Hello, Worl...|  8|
|7282405|Sentence|5691624|&quot;Hello, Worl...|  9|
|7282406|Sentence|5691624|&quot;Hello, Worl...| 10|
|7282407|Sentence|5691624|&quot;Hello, Worl...| 11|
|7282408|Sentence|5691624|&quot;Hello, Worl...| 12|
|7282409|Sentence|5691624|&quot;Hello, Worl...| 13|
|7282410|Sentence|5691624|&quot;Hello, Worl...| 14|
|7282411|Sentence|5691624|&quot;Hello, Worl...| 15|
|7282412|Sentence|5691624|&quot;Hello, Worl...| 16|
|7282413|Sen

In [198]:
from pyspark.sql.functions import isnan

In [222]:
clause_verticesTextJ.filter((clause_verticesTextJ["src_id"] == "") | clause_verticesTextJ["src_id"].isNull() | isnan(clause_verticesTextJ["src_id"])).show()

+---+--------+-----+-----+-----+------+
| id|nodeType|attr1|attr2|attr3|src_id|
+---+--------+-----+-----+-----+------+
+---+--------+-----+-----+-----+------+



In [231]:
conf = SparkConf()
conf.set("spark.sql.crossJoin.enabled", "true")  # 避免Spark SQL在Linux执行成功，而放在自开发平台上失败的问题

In [238]:
from pyspark.sql import SparkSession

In [240]:
SparkSession.builder.config("spark.sql.crossJoin.enabled", "true")

In [245]:
pyspark.sql.crossJoin.enabled=true

NameError: name 'true' is not defined

In [241]:
clause_verticesText=clause_verticesTextJ.join(sen_verticesText.select("id","did","dtitle","sid"), sen_verticesText.id==clause_verticesTextJ.src_id, "inner")

In [243]:
clause_verticesText.show()

AnalysisException: u'Detected implicit cartesian product for INNER join between logical plans\nProject [id#1842, nodeType#1836, attr1#1848, attr2#1854, attr3#1860, src_id#1288]\n+- Join Inner, ((id#1842 = src_id#1288) && (id#1842 = dst_id#1429))\n   :- Project [_c1#559 AS id#1842, _c0#558 AS nodeType#1836, _c2#560 AS attr1#1848, _c3#561 AS attr2#1854, _c4#562 AS attr3#1860]\n   :  +- Filter ((isnotnull(_c0#558) && (_c0#558 = Clause)) && isnotnull(_c1#559))\n   :     +- Relation[_c0#558,_c1#559,_c2#560,_c3#561,_c4#562,_c5#563] csv\n   +- Project [src_id#1288, id#700 AS dst_id#1429]\n      +- Join Inner, ((src_id#1288 = id#700) && (dst#892 = id#700))\n         :- Project [dst#892, id#700 AS src_id#1288]\n         :  +- Join Inner, (src#871 = id#700)\n         :     :- Project [translate(_c0#852, Edge(, ) AS src#871, _c1#853 AS dst#892]\n         :     :  +- Filter (isnotnull(translate(_c0#852, Edge(, )) && isnotnull(_c1#853))\n         :     :     +- Relation[_c0#852,_c1#853,_c2#854] csv

In [219]:
# clause_verticesText=clause_verticesTextJ.join(sen_verticesText.select("*"), clause_verticesTextJ.src_id==sen_verticesText.id, "outer")

In [108]:
##did doc->sentence
# edgesText2 = edgesText1.join(verticesText1.select("id","nodeType"), edgesText1.src == verticesText1.select("id","nodeType").id, "inner")
did_verticesText1=verticesText1.join(edgesText3.filter("label = 'contains the sentence'").select("dst_id", "dst_nodeType"),verticesText1.id==edgesText3.dst_id, "inner")

In [110]:
did_verticesText1.show(10)

+-------+--------+-----+--------------------+--------------------+---+------+---+------+-------+------+------------+-------+------------+
|     id|nodeType|attr1|               attr2|               attr3|did|dtitle|sid|weight|matched|src_id|src_nodeType| dst_id|dst_nodeType|
+-------+--------+-----+--------------------+--------------------+---+------+---+------+-------+------+------------+-------+------------+
|1014942|Sentence|  102|Both the process ...|              Genome|  0|      |  0|     0|       |626404|    Document|1014942|    Sentence|
|1015263|Sentence|   99|In the gas phase,...|             Alcohol|  0|      |  0|     0|       |626407|    Document|1015263|    Sentence|
|1015551|Sentence|    5|In the 19th centu...|             Cartoon|  0|      |  0|     0|       |626411|    Document|1015551|    Sentence|
|1015552|Sentence|    6|In the early 20th...|             Cartoon|  0|      |  0|     0|       |626411|    Document|1015552|    Sentence|
|1015610|Sentence|   64|Thomson ca

## 2. Load Edges

In [80]:
edgesText0 = spark.read.csv(File_edgesTextRDD)

In [81]:
edgesText0.show(10)

+------------+-------+--------------------+
|         _c0|    _c1|                 _c2|
+------------+-------+--------------------+
|Edge(5691617|7284099|contains the sent...|
|Edge(5691617|7284100|contains the sent...|
|Edge(5691617|7284101|contains the sent...|
|Edge(5691617|7284102|contains the sent...|
|Edge(5691617|7284103|contains the sent...|
|Edge(5691617|7284104|contains the sent...|
|Edge(5691617|7284105|contains the sent...|
|Edge(5691617|7284106|contains the sent...|
|Edge(5691617|7284107|contains the sent...|
|Edge(5691617|7284108|contains the sent...|
+------------+-------+--------------------+
only showing top 10 rows



In [82]:
edgesText0=edgesText0.select(f.translate(f.col("_c0"), "Edge(", "").alias("src"), "_c1", f.translate(f.col("_c2"), ")", "").alias("label"))

In [83]:
edgesText0.show()

+-------+-------+--------------------+
|    src|    _c1|               label|
+-------+-------+--------------------+
|5691617|7284099|contains the sent...|
|5691617|7284100|contains the sent...|
|5691617|7284101|contains the sent...|
|5691617|7284102|contains the sent...|
|5691617|7284103|contains the sent...|
|5691617|7284104|contains the sent...|
|5691617|7284105|contains the sent...|
|5691617|7284106|contains the sent...|
|5691617|7284107|contains the sent...|
|5691617|7284108|contains the sent...|
|5691617|7284109|contains the sent...|
|5691617|7284110|contains the sent...|
|5691617|7284111|contains the sent...|
|5691617|7284112|contains the sent...|
|5691617|7284113|contains the sent...|
|5691617|7284114|contains the sent...|
|5691617|7284115|contains the sent...|
|5691617|7284116|contains the sent...|
|5691617|7284117|contains the sent...|
|5691617|7284118|contains the sent...|
+-------+-------+--------------------+
only showing top 20 rows



In [84]:
edgesText1 = edgesText0.select("*").withColumnRenamed("_c1", "dst")

In [85]:
edgesText1.show()

+-------+-------+--------------------+
|    src|    dst|               label|
+-------+-------+--------------------+
|5691617|7284099|contains the sent...|
|5691617|7284100|contains the sent...|
|5691617|7284101|contains the sent...|
|5691617|7284102|contains the sent...|
|5691617|7284103|contains the sent...|
|5691617|7284104|contains the sent...|
|5691617|7284105|contains the sent...|
|5691617|7284106|contains the sent...|
|5691617|7284107|contains the sent...|
|5691617|7284108|contains the sent...|
|5691617|7284109|contains the sent...|
|5691617|7284110|contains the sent...|
|5691617|7284111|contains the sent...|
|5691617|7284112|contains the sent...|
|5691617|7284113|contains the sent...|
|5691617|7284114|contains the sent...|
|5691617|7284115|contains the sent...|
|5691617|7284116|contains the sent...|
|5691617|7284117|contains the sent...|
|5691617|7284118|contains the sent...|
+-------+-------+--------------------+
only showing top 20 rows



In [ ]:
verticesText1J=verticesText1

In [86]:
# edgesText1.join(verticesText1.set_index('id'), on='src')
# edgesText1.join(verticesText1, ["src"], "outer")
# edgesText2 = edgesText1.join(verticesText1, edgesText1.src == verticesText1.id, "inner")

In [91]:
edgesText2 = edgesText1.join(verticesText1.select("id","nodeType"), edgesText1.src == verticesText1.select("id","nodeType").id, "inner")

In [98]:
edgesText2=edgesText2.withColumnRenamed("id", "src_id").withColumnRenamed("nodeType", "src_nodeType")

In [99]:
edgesText2.show(10)

+-------+-------+--------------------+-------+------------+
|    src|    dst|               label| src_id|src_nodeType|
+-------+-------+--------------------+-------+------------+
| 100010|2216143|contains the mention| 100010|      Clause|
| 100010|2216144|contains the mention| 100010|      Clause|
| 100140|3579975|contains the mention| 100140|      Clause|
| 100140|3579976|contains the mention| 100140|      Clause|
|1002011|2086735|contains the mention|1002011|      Clause|
|1002011|2086736|contains the mention|1002011|      Clause|
|1002185|3571984|contains the mention|1002185|      Clause|
|1002185|3571985|contains the mention|1002185|      Clause|
| 100227|1679175|contains the mention| 100227|      Clause|
| 100227|1679176|contains the mention| 100227|      Clause|
+-------+-------+--------------------+-------+------------+
only showing top 10 rows



In [101]:
edgesText3 = edgesText2.join(verticesText1.select("id","nodeType"), edgesText2.dst == verticesText1.select("id","nodeType").id, "inner")

In [105]:
edgesText3=edgesText3.withColumnRenamed("id", "dst_id").withColumnRenamed("nodeType", "dst_nodeType")

In [106]:
edgesText3.show(10)

+-------+-------+-------------------+-------+------------+-------+------------+
|    src|    dst|              label| src_id|src_nodeType| dst_id|dst_nodeType|
+-------+-------+-------------------+-------+------------+-------+------------+
|1234113| 100010|contains the clause|1234113|    Sentence| 100010|      Clause|
|3875588|1000240|contains the clause|3875588|    Sentence|1000240|      Clause|
|3650624|1000280|contains the clause|3650624|    Sentence|1000280|      Clause|
|2068926|1000665|contains the clause|2068926|    Sentence|1000665|      Clause|
| 467648|1000795|contains the clause| 467648|    Sentence|1000795|      Clause|
| 233944|1000839|contains the clause| 233944|    Sentence|1000839|      Clause|
|4022932|1000888|contains the clause|4022932|    Sentence|1000888|      Clause|
|1212133| 100140|contains the clause|1212133|    Sentence| 100140|      Clause|
| 452611|1001866|contains the clause| 452611|    Sentence|1001866|      Clause|
|1487892|1002011|contains the clause|148

In [107]:
edgesText3.filter("label = 'contains the sentence'").show()

+------+-------+--------------------+------+------------+-------+------------+
|   src|    dst|               label|src_id|src_nodeType| dst_id|dst_nodeType|
+------+-------+--------------------+------+------------+-------+------------+
|626404|1014942|contains the sent...|626404|    Document|1014942|    Sentence|
|626407|1015263|contains the sent...|626407|    Document|1015263|    Sentence|
|626411|1015551|contains the sent...|626411|    Document|1015551|    Sentence|
|626411|1015552|contains the sent...|626411|    Document|1015552|    Sentence|
|626411|1015610|contains the sent...|626411|    Document|1015610|    Sentence|
|626413|1015827|contains the sent...|626413|    Document|1015827|    Sentence|
|626414|1015968|contains the sent...|626414|    Document|1015968|    Sentence|
|626417|1016299|contains the sent...|626417|    Document|1016299|    Sentence|
|626417|1016518|contains the sent...|626417|    Document|1016518|    Sentence|
|626417|1016816|contains the sent...|626417|    Docu

In [27]:
verticesText1.filter("nodeType = 'Clause'").show()##remember here the  attr3 is not complete, however, this data will not be used.

+----+--------+--------------------+-----+--------------------+
|  id|nodeType|               attr1|attr2|               attr3|
+----+--------+--------------------+-----+--------------------+
|4559|  Clause|("A significant c...|  SVA|A significant con...|
|4560|  Clause|("by the French r...|  SVO|by the French res...|
|4561|  Clause|("quinine and str...|  SVC|quinine and stryc...|
|4562|  Clause|("strychnine", "i...|  SVC|          strychnine|
|4563|  Clause|("Some writers", ...|  SVO|        Some writers|
|4564|  Clause|("those", "would ...|  SVO|               those|
|4565|  Clause|("The English Cha...|  SVC|The English Chann...|
|4566|  Clause|("Johnston", "res...|  SVO|            Johnston|
|4567|  Clause|("Tupper", "becam...|  SVO|              Tupper|
|4568|  Clause|("His father", "m...|  SVA|          His father|
|4569|  Clause|("Many physicists...|  SVA|     Many physicists|
|4570|  Clause|("of quantum mech...|  SVC|of quantum mechanics|
|4571|  Clause|("The King", "app...|  SV

## 3. Create Graph & Test

In [28]:
Graph = GraphFrame(verticesText1, edgesText1)

In [29]:
Graph.edges.show(10)

+-------+-------+--------------------+
|    src|    dst|               label|
+-------+-------+--------------------+
|5691617|7284099|contains the sent...|
|5691617|7284100|contains the sent...|
|5691617|7284101|contains the sent...|
|5691617|7284102|contains the sent...|
|5691617|7284103|contains the sent...|
|5691617|7284104|contains the sent...|
|5691617|7284105|contains the sent...|
|5691617|7284106|contains the sent...|
|5691617|7284107|contains the sent...|
|5691617|7284108|contains the sent...|
+-------+-------+--------------------+
only showing top 10 rows



In [30]:
Graph.vertices.show(10)

+---+--------+-----+--------------------+-------------------+
| id|nodeType|attr1|               attr2|              attr3|
+---+--------+-----+--------------------+-------------------+
|  0|Sentence|    1|Politics of Burun...|Politics of Burundi|
|  1|Sentence|    2|Executive power i...|Politics of Burundi|
|  2|Sentence|    3|Legislative power...|Politics of Burundi|
|  3|Sentence|    4|The political lan...|Politics of Burundi|
|  4|Sentence|    5|The current Presi...|Politics of Burundi|
|  5|Sentence|    6|Nkurunziza was th...|Politics of Burundi|
|  6|Sentence|    7|In November 1995,...|Politics of Burundi|
|  7|Sentence|    8|In July 1996, for...|Politics of Burundi|
|  8|Sentence|    9|He declared himse...|Politics of Burundi|
|  9|Sentence|   10|Widespread condem...|Politics of Burundi|
+---+--------+-----+--------------------+-------------------+
only showing top 10 rows



In [31]:
Graph.vertices.count()

8593092

In [32]:
Graph.edges.count()

11607488

In [33]:
# Example 1: bfs Algorithm
filteredPaths = Graph.bfs(
  fromExpr = "id = '5691617'",
  toExpr = "id = '7284108'",
  maxPathLength = 2)

In [34]:
# display(filteredPaths)
filteredPaths.show()

+--------------------+--------------------+--------------------+
|                from|                  e0|                  to|
+--------------------+--------------------+--------------------+
|[5691617, Documen...|[5691617, 7284108...|[7284108, Sentenc...|
+--------------------+--------------------+--------------------+

